In [1]:
#ADX
#On Balance Volume
#VWAP

In [1]:
from pandas_datareader import data as wb

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
import pandas as pd
import numpy as np

In [56]:
ticker = 'TSLA'
df = wb.DataReader(ticker, data_source = 'yahoo', start = '2011-12-1', end = '2020-7-30')

In [57]:
df.to_json('tslafin.json')

In [58]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2011-12-01,33.990002,31.980000,32.570000,32.599998,1030200,32.599998
2011-12-02,33.689999,32.400002,32.830002,33.299999,802800,33.299999
2011-12-05,35.000000,33.430000,33.529999,34.419998,1160100,34.419998
2011-12-06,34.980000,34.029999,34.200001,34.869999,951800,34.869999
2011-12-07,34.889999,33.799999,34.630001,34.189999,674300,34.189999
...,...,...,...,...,...,...
2020-07-20,1650.000000,1488.000000,1519.010010,1643.000000,17121400,1643.000000
2020-07-21,1675.000000,1558.000000,1639.930054,1568.359985,16157300,1568.359985
2020-07-22,1626.420044,1562.000000,1599.000000,1592.329956,14161100,1592.329956


In [43]:
def vwap(df):
    q = df.Volume.values
    p = df['Adj Close'].values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())

In [44]:
vwap(df)

,High,Low,Open,Close,Volume,Adj Close,vwap
Date,,,,,,,
2011-12-01,33.990002,31.980000,32.570000,32.599998,1030200,32.599998,32.599998
2011-12-02,33.689999,32.400002,32.830002,33.299999,802800,33.299999,32.906578
2011-12-05,35.000000,33.430000,33.529999,34.419998,1160100,34.419998,33.493167
2011-12-06,34.980000,34.029999,34.200001,34.869999,951800,34.869999,33.825360
2011-12-07,34.889999,33.799999,34.630001,34.189999,674300,34.189999,33.878589
...,...,...,...,...,...,...,...
2020-07-20,1650.000000,1488.000000,1519.010010,1643.000000,17121400,1643.000000,324.971621
2020-07-21,1675.000000,1558.000000,1639.930054,1568.359985,16157300,1568.359985,326.313555
2020-07-22,1626.420044,1562.000000,1599.000000,1592.329956,14161100,1592.329956,327.509968


In [45]:
df['VWAP'] = np.cumsum(df['Volume'] * (df['High'] + df['Low'] + df['Adj Close'])/3) / np.cumsum(df['Volume'])

In [46]:
import plotly.graph_objects as go

In [47]:
fig = go.Figure()

fig = go.Figure(data=[go.Candlestick(x = df.index,
                                     open = df['Open'],
                                     high = df['High'],
                                     low = df['Low'],
                                     showlegend = False,
                                     close = df['Adj Close'])])

fig.add_trace(go.Scatter(x=df.index, y=df['VWAP'], name="VWAP",
                     line_color='cornflowerblue', mode='lines',
        line={'dash': 'dash'}, marker_line_width=2,
        marker_size=10,
        opacity = 0.8))

fig.update_layout(title_text= 'Volume Weighted Average Price', width=1000,
                height=600, xaxis_rangeslider_visible=True)

In [48]:
#ADX(Average Directional Index)

#ADX is a way of measuring the strength of a trend

#Values range from 0 to 100 and quantifies the strength of a trend as per below:
#0 -25 Absent or Weak trend
#25 -50 Strong Trend
#50 -75 Very Strong Trend
#75 -100 Extremely Strong Trend

#ADX is non directional meaning the ADX value makes no inference about the direction of the trend but only about the 
#strength of the trend

#The Calcualtion involves finding both the positive and negative directional movement (by compariing successive highs
#and successive lows) and then calculating the smoothed average of the difference of these 

In [49]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2


def ADX(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2[['ADX', 'DIplusN', 'DIminusN']]

In [50]:
adx_plot = ADX(df,14)

In [51]:
import plotly.graph_objects as go


In [52]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=adx_plot.index, y=adx_plot['ADX'], name="Average Directional Index",
                 line_color='cornflowerblue', opacity = 0.5))

fig.add_trace(go.Scatter(x=adx_plot.index, y=adx_plot['DIplusN'], name="Plus Directional Index",
                 line_color='darkseagreen', opacity = 0.5))

fig.add_trace(go.Scatter(x=adx_plot.index, y=adx_plot['DIminusN'], name="Minus Directional Index",
                 line_color='mediumvioletred', opacity = 0.5))

fig.update_layout(title_text='Tesla Average Directional Index(ADX)', width=1000,
height=600, xaxis_rangeslider_visible=True)
fig.show()
                

In [53]:
#OBV is a momentum indicator which uses changes in trading volume as an indicator future asset price moves

#OBV formulation based on the theory that volume precedes price movement.  A rising OBV reflects positive volumen pressure
#that can lead to higher prices and falling OBV predicts decline in prices 

#Leading market indicator put prone to making false signals.  Typically use in conjunction with 
#lagging indicators such as MACD

#the calculation of OBV is fairly straightforward and it simply the cumulative sum of volume traded
#adjusted for the direction of the corresponding asset price move

In [54]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']

In [22]:
df['OBV'] = OBV(df)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
SMA100 = pd.DataFrame()

SMA100['OBV'] = df['OBV'].rolling(window = 100).mean()

In [37]:
SMA100

,OBV
Date,
2011-12-01,NaN
2011-12-02,NaN
2011-12-05,NaN
2011-12-06,NaN
2011-12-07,NaN
...,...
2020-07-14,595646299.0
2020-07-15,596742110.0
2020-07-16,597846835.0


In [39]:
fig = go.Figure()

fig = go.Figure(data=[go.Candlestick(x = df.index,
                                     open = df['Open'],
                                     high = df['High'],
                                     low = df['Low'],
                                     showlegend = False,
                                     close = df['Adj Close'])])
fig.show()

In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = SMA100.index, y = SMA100['OBV'], name="SMA 100 OBV",
                 line_color='blue', opacity = 0.8))

fig.add_trace(go.Scatter(x=df.index, y=df['OBV'], name="On Balance Volume",
                 line_color='cornflowerblue', opacity = 0.5))
fig.update_layout(title_text='Tesla On balance Volume', width=1000,
height=600, xaxis_rangeslider_visible=True)
fig.show()